In [15]:
num_sample = 300
num_burn = 20
sample_size = 800
n_cpu = 10

min_degree = 1
max_degree = 1

from dgp import sample_network_chain, sample_Y1, get_graph, sample_Y2, agcEffect
import numpy as np

# # 1. Simulate or load data
def get_graph(N, min_degree=2, max_degree=5, seed=0):
    """
    Generate a symmetric adjacency matrix of an undirected graph where each node has:
      - at least `min_degree` neighbors
      - at most `max_degree` neighbors
    """
    np.random.seed(seed)
    adj_matrix = np.zeros((N, N), dtype=int)
    degree = np.zeros(N, dtype=int)
    degrees = np.random.randint(min_degree, max_degree + 1, size=N)

    # Candidate edges (i < j to avoid duplicates in symmetric matrix)
    candidates = [(i, j) for i in range(N) for j in range(i + 1, N)]
    np.random.shuffle(candidates)

    for (i, j) in candidates:
        if degree[i] < degrees[i] and degree[j] < degrees[j]:
            adj_matrix[i, j] = 1
            adj_matrix[j, i] = 1
            degree[i] += 1
            degree[j] += 1

    # Fix nodes with degree < min_degree by connecting to random nodes
    for i in range(N):
        while degree[i] < degrees[i]:
            # Potential new connections: not self, not already connected, target degree < max
            possible = [j for j in range(N)
                        if j != i and adj_matrix[i, j] == 0 and degree[j] < max_degree]
            if not possible:
                break  # cannot fix further due to global constraints
            j = np.random.choice(possible)
            adj_matrix[i, j] = 1
            adj_matrix[j, i] = 1
            degree[i] += 1
            degree[j] += 1

    return adj_matrix


adj = get_graph(sample_size, min_degree, max_degree, seed=1)
print(adj.sum(axis=1).min(), adj.sum(axis=1).max())

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
beta = np.array([-1*min_degree,2,-0.2,2,0.1,-2,0.1,2,0.1,0])  # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0, sample_Y_func=sample_Y1, Atype=('gen', 0.7))

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

dir, dir2, dir3, dir4 = [], [], [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(np.mean(Y[A==1]) - Y[A==0]))
    dir2.append(np.mean(A))
    dir3.append(np.mean(A[L[:,0]==1]) - np.mean(A[L[:,0]==0]))
    dir4.append(np.mean(Y[L[:,0]==1]) - np.mean(Y[L[:,0]==0]))

np.mean(dir), np.mean(dir2), np.mean(Y_chain), np.mean(dir3), np.mean(dir4)

1 1


100%|██████████| 320/320 [00:14<00:00, 22.42it/s]


(0.626430863757238,
 0.3835124999999999,
 0.476475,
 0.3409563607062833,
 0.37728303309835154)

In [16]:
from drnet_em import *

idx = 0
ret_em = doubly_robust_em(A_chain[idx], L_chain[idx], Y_chain[idx], adj, return_raw=True)

In [17]:
from drnet import *

idx = 0
ret_dr = doubly_robust(A_chain[idx], L_chain[idx], Y_chain[idx], adj, return_raw=True)

In [18]:
ret_dr.keys()

dict_keys(['psi_gamma', 'psi_zero', 'psi_1_gamma', 'psi_0_gamma'])

In [19]:
key = 'psi_gamma'
ret_em[key].mean(), ret_em[key].std(), '|', ret_dr[key].mean(), ret_dr[key].std()

(0.6047936155424195,
 1.4539741716302226,
 '|',
 0.6001210287326834,
 1.4448836388187571)

In [20]:
key = 'psi_1_gamma'
ret_em[key].mean(), ret_em[key].std(), '|', ret_dr[key].mean(), ret_dr[key].std()

(0.7281734142241624,
 1.9663221915185491,
 '|',
 0.7270518742728167,
 1.9765832019004503)

In [21]:
key = 'psi_0_gamma'
ret_em[key].mean(), ret_em[key].std(), '|', ret_dr[key].mean(), ret_dr[key].std()

(0.31690741861835287,
 1.5020230250032185,
 '|',
 0.31584878011131606,
 1.5115915879487773)

In [22]:
key = 'psi_zero'
ret_em[key].mean(), ret_em[key].std(), '|', ret_dr[key].mean(), ret_dr[key].std()

(0.3577799702921217,
 1.1408136634433637,
 '|',
 0.3577799702921217,
 1.1408136634433637)

In [4]:
from drnet_em import *

def get_numerator_pi_em(a_mat, A, GL, neighbours, neighbours_2hop, gamma, adj_matrix):
    N = adj_matrix.shape[0]
    num_rep = a_mat.shape[1]
    
    max_neighbours = adj_matrix.sum(axis=1).max()
    numerator = np.zeros((N, 2, max_neighbours+1))
    for i in range(N):
        ni = [i]+neighbours[i]
        vec_n = a_mat[ni].copy()
        
        # compute outter product
        for a in [0, 1]:
            vec_n[0] = a
            mat_n = np.einsum('ik,jk->ijk', vec_n, vec_n)
            adj_max_n = adj_matrix[ni, :][:, ni]
            aa_n_i = (mat_n * adj_max_n[:, :, None]).sum(axis=(0, 1))/2
            
            nout = neighbours_2hop[i]
            vec_n_out = A[nout] # 790 by 1
            mat_n_out = np.einsum('ik,j->ijk', vec_n, vec_n_out)
            adj_max_n_out = adj_matrix[ni, :][:, nout]
            aa_out_i = (mat_n_out * adj_max_n_out[:, :, None]).sum(axis=(0, 1))
            
            GL_neighbour_i = GL[ni].dot(vec_n)
            num_vec_i = np.exp(GL_neighbour_i + gamma[7]*aa_n_i + gamma[7]*aa_out_i)
            
            num_neighbours = len(neighbours[i])
            vec_treated_neighbours = vec_n[1:].sum(axis=0)
            for g in range(num_neighbours+1):
                # average over reps with num_neigh==g
                numerator[i, a, g] = np.sum(num_vec_i[vec_treated_neighbours==g])/num_rep*(2**num_neighbours)
    return numerator

def get_numerator_pi_em_new(a_mat_dict, A, GL, neighbours, neighbours_2hop, gamma, adj_matrix):
    N = adj_matrix.shape[0]
    
    max_neighbours = adj_matrix.sum(axis=1).max()
    numerator = np.zeros((N, 2, max_neighbours+1))
    for i in range(N):
        ni = [i]+neighbours[i]
        a_mat = a_mat_dict[len(neighbours[i])]
        vec_n = np.zeros((len(ni), a_mat.shape[1]))
        vec_n[1:, :] = a_mat.copy()
        
        # compute outter product
        for a in [0, 1]:
            vec_n[0] = a
            mat_n = np.einsum('ik,jk->ijk', vec_n, vec_n)
            adj_max_n = adj_matrix[ni, :][:, ni]
            aa_n_i = (mat_n * adj_max_n[:, :, None]).sum(axis=(0, 1))/2
            
            nout = neighbours_2hop[i]
            vec_n_out = A[nout] # 790 by 1
            mat_n_out = np.einsum('ik,j->ijk', vec_n, vec_n_out)
            adj_max_n_out = adj_matrix[ni, :][:, nout]
            aa_out_i = (mat_n_out * adj_max_n_out[:, :, None]).sum(axis=(0, 1))
            
            GL_neighbour_i = GL[ni].dot(vec_n)
            num_vec_i = np.exp(GL_neighbour_i + gamma[7]*aa_n_i + gamma[7]*aa_out_i)
            
            num_neighbours = len(neighbours[i])
            vec_treated_neighbours = vec_n[1:].sum(axis=0)
            for g in range(num_neighbours+1):
                # average over reps with num_neigh==g
                numerator[i, a, g] = np.sum(num_vec_i[vec_treated_neighbours==g])
    return numerator

def doubly_robust_em(A, L, Y, adj_matrix, treatment_allocation=0.7, num_rep=2000, seed=1, return_raw=False,
                  mispec=None):
    np.random.seed(seed)
    
    # fit models
    L_a, L_y = L.copy(), L.copy()
    if mispec == 'outcome':
        L_y = np.random.binomial(1, 0.5, size=L_y.shape)
    elif mispec == 'treatment':
        L_a = np.random.binomial(1, 0.5, size=L_a.shape)
    X_y = build_design_matrix_Y(A, L_y, Y, adj_matrix)
    model_y = fit_logistic_model(X_y, Y)
    X_a = build_design_matrix_A(L_a, A, adj_matrix)
    model_a = fit_logistic_model(X_a, A)
    
    # compute pi
    gamma = np.concatenate([model_a.intercept_, model_a.coef_.flatten()])
    N = adj_matrix.shape[0]
    neighbours, neighbours_2hop = get_2hop_neighbors(adj_matrix)
    L_nb = get_neighbor_summary(L_a, adj_matrix)
    GL = gamma[0] + L.dot(np.array([gamma[1], gamma[3], gamma[5]])) \
        + L_nb.dot(np.array([gamma[2], gamma[4], gamma[6]]))
        
    denominator = get_norm_constant_new(A, GL, neighbours, neighbours_2hop, gamma, adj_matrix)
    print(denominator)
    
    # compute the influence function
    #a_mat = np.random.binomial(1, 0.5, size=(Y.shape[0], num_rep))
    a_mat_dict = generate_all_binary_vectors(adj_matrix.sum(axis=1).max())
    numerator_vec = get_numerator_pi_em_new(a_mat_dict, A, GL, neighbours, neighbours_2hop, gamma, adj_matrix)
    pi_vec = numerator_vec / denominator[:, None, None] # N by 2 by max_neighbours

    beta_hat_vec = np.zeros(pi_vec.shape)
    for a in [0, 1]:
        for g in range(pi_vec.shape[2]):
            X_y_eval = X_y.copy()
            X_y_eval[:, 0] = a
            X_y_eval[:, 1] = g
            beta_hat_vec[:, a, g] = model_y.predict_proba(X_y_eval)[:, 1]

    A_nb = get_neighbor_summary(A.reshape(-1, 1), adj_matrix).flatten()
    print(((A == 0) & (A_nb == 0)).sum(), Y[(A == 0) & (A_nb == 0)])
    psi_vec = np.zeros(pi_vec.shape)
    for a in [0, 1]:
        for g in range(pi_vec.shape[2]):
            I = ((A == a) & (A_nb == g)).astype(int)
            pi = pi_vec[:, a, g].copy()
            pi[I==0] = 1
            beta_hat = beta_hat_vec[:, a, g].copy() * 0
            psi_vec[:,a,g] = beta_hat + I / pi * (Y - beta_hat)

    # compute all 1
    prob_allocation_vec = np.zeros((N, pi_vec.shape[2]))
    for i in range(N):
        num_neighbours = len(neighbours[i])
        prob_allocation_vec[i,:num_neighbours+1] = np.array([binom.pmf(k, num_neighbours, treatment_allocation) 
                                             for k in range(num_neighbours+1)])

    psi_gamma, psi_1_gamma, psi_0_gamma, psi_zero = np.zeros(N), np.zeros(N), np.zeros(N), np.zeros(N)
    psi_1_gamma = (psi_vec[:, 1, :]*prob_allocation_vec).sum(axis=1)
    psi_0_gamma = (psi_vec[:, 0, :]*prob_allocation_vec).sum(axis=1)
    psi_zero = psi_vec[:,0,0]
    psi_gamma = treatment_allocation * psi_1_gamma + (1-treatment_allocation) * psi_0_gamma

    if return_raw:
        return {
            'psi_gamma': psi_gamma,
            'psi_zero': psi_zero,
            'psi_1_gamma': psi_1_gamma,
            'psi_0_gamma': psi_0_gamma,
        }

    # Compute effects
    avg_psi_gamma = np.mean(psi_gamma)
    direct_effect = np.mean(psi_1_gamma) - np.mean(psi_0_gamma)
    spillover_effect = np.mean(psi_0_gamma) - np.mean(psi_zero)

    return {
        "average": avg_psi_gamma,
        "direct_effect": direct_effect,
        "spillover_effect": spillover_effect,
        "psi_1_gamma": np.mean(psi_1_gamma),
        "psi_0_gamma": np.mean(psi_0_gamma),
        "psi_zero": np.mean(psi_zero),
    }

In [5]:
from drnet import *

def doubly_robust(A, L, Y, adj_matrix, treatment_allocation=0.7, num_rep=1000, seed=1, return_raw=False, psi_0_gamma_only=False,
                  mispec=None):
    np.random.seed(seed)
    
    # fit models
    L_a, L_y = L.copy(), L.copy()
    if mispec == 'outcome':
        L_y = np.random.binomial(1, 0.5, size=L_y.shape)
    elif mispec == 'treatment':
        L_a = np.random.binomial(1, 0.5, size=L_a.shape)
    X_y = build_design_matrix_Y(A, L_y, Y, adj_matrix)
    model_y = fit_logistic_model(X_y, Y)
    X_a = build_design_matrix_A(L_a, A, adj_matrix)
    model_a = fit_logistic_model(X_a, A)
    
    # compute pi
    gamma = np.concatenate([model_a.intercept_, model_a.coef_.flatten()])
    N = adj_matrix.shape[0]
    neighbours, neighbours_2hop = get_2hop_neighbors(adj_matrix)
    L_nb = get_neighbor_summary(L_a, adj_matrix)
    GL = gamma[0] + L.dot(np.array([gamma[1], gamma[3], gamma[5]])) \
        + L_nb.dot(np.array([gamma[2], gamma[4], gamma[6]]))
        
    denominator = get_norm_constant(A, GL, neighbours, neighbours_2hop, gamma, adj_matrix)
    
    # compute the influence function
    a_mat = np.random.binomial(1, treatment_allocation, size=(Y.shape[0], num_rep))
    if psi_0_gamma_only is False:
        numerator_vec, I = get_numerator_pi_vec(a_mat, A, GL, neighbours, neighbours_2hop, gamma, adj_matrix, Atype='all')
        pi_vec = numerator_vec / denominator[:, None]
        psi_gamma = np.zeros((N, num_rep))
        for i in range(num_rep):
            X_y_eval = build_design_matrix_Y(a_mat[:,i], L_y, Y, adj_matrix)
            beta_hat = compute_beta_probs(X_y_eval, model_y, Atype='all') * 0
            w = I[:,i] / pi_vec[:, i]
            #w[pi_vec[:, i]<1e-3] = 0
            w_norm = w#/np.sum(w)*N if np.sum(w) > 0 else 0
            psi = beta_hat + w_norm * (Y - beta_hat)
            psi_gamma[:, i] = psi.copy()
        
        numerator, I = get_numerator_pi_vec(a_mat, A, GL, neighbours, neighbours_2hop, gamma, adj_matrix, Atype='ind_treat_1')
        pi_1_vec = numerator / denominator[:, None]
        psi_1_gamma = np.zeros((N, num_rep))
        for i in range(num_rep):
            X_y_eval = build_design_matrix_Y(a_mat[:,i], L_y, Y, adj_matrix)
            beta_hat = compute_beta_probs(X_y_eval, model_y, Atype='ind_treat_1') * 0
            w = I[:,i] / pi_1_vec[:, i]
            #w[pi_1_vec[:, i]<1e-3] = 0
            w_norm = w#/np.sum(w)*N if np.sum(w) > 0 else 0
            psi = beta_hat + w_norm * (Y - beta_hat)
            psi_1_gamma[:, i] = psi.copy()
    else:
        psi_gamma = np.zeros((N, num_rep))
        psi_1_gamma = np.zeros((N, num_rep))
    
    numerator, I = get_numerator_pi_vec(a_mat, A, GL, neighbours, neighbours_2hop, gamma, adj_matrix, Atype='ind_treat_0')
    pi_0_vec = numerator / denominator[:, None]
    psi_0_gamma = np.zeros((N, num_rep))
    for i in range(num_rep):
        X_y_eval = build_design_matrix_Y(a_mat[:,i], L_y, Y, adj_matrix)
        beta_hat = compute_beta_probs(X_y_eval, model_y, Atype='ind_treat_0') * 0
        w = I[:,i] / pi_0_vec[:, i]
        #w[pi_0_vec[:, i]<1e-3] = 0
        w_norm = w#/np.sum(w)*N if np.sum(w) > 0 else 0
        psi = beta_hat + w_norm * (Y - beta_hat)
        psi_0_gamma[:, i] = psi.copy()
    
    if psi_0_gamma_only is False:
        a_mat = np.zeros((Y.shape[0],1))
        numerator, I = get_numerator_pi_vec(a_mat, A, GL, neighbours, neighbours_2hop, gamma, adj_matrix, Atype='all_0')
        pi_zero_vec = numerator / denominator[:, None]
        psi_zero = np.zeros((N,))
        X_y_eval = build_design_matrix_Y(a_mat, L_y, Y, adj_matrix)
        beta_hat = compute_beta_probs(X_y_eval, model_y, Atype='all_0') * 0
        w = I[:,0] / pi_zero_vec[:, 0] 
        #w[pi_zero_vec[:, 0]<1e-3] = 0
        w_norm = w#/np.sum(w)*N if np.sum(w) > 0 else 0
        psi = beta_hat + w_norm * (Y - beta_hat)
        psi_zero = psi.copy()
    else:
        psi_zero = np.zeros((N,))
    
    # Compute effects
    avg_psi_gamma = np.mean(psi_gamma)
    direct_effect = np.mean(psi_1_gamma) - np.mean(psi_0_gamma)
    spillover_effect = np.mean(psi_0_gamma) - np.mean(psi_zero)

    if return_raw:
        return {
            'psi_gamma': np.mean(psi_gamma, axis=1),
            'psi_zero': psi_zero,
            'psi_1_gamma': np.mean(psi_1_gamma, axis=1),
            'psi_0_gamma': np.mean(psi_0_gamma, axis=1),
        }

    # print("psi_zero:", psi_zero)
    # print("beta_hat:", beta_hat.mean())
    # print("psi_0_gamma:", np.mean(psi_0_gamma))
    # print("psi_1_gamma:", np.mean(psi_1_gamma))
    # print("average:", np.mean(psi_gamma))
    # print("direct_effect:", direct_effect)
    # print("spillover_effect:", spillover_effect)
    
    return {
        "average": avg_psi_gamma,
        "direct_effect": direct_effect,
        "spillover_effect": spillover_effect,
        "psi_1_gamma": np.mean(psi_1_gamma),
        "psi_0_gamma": np.mean(psi_0_gamma),
        "psi_zero": np.mean(psi_zero),
    }

In [ ]:
doubly_robust(A_chain[idx], L_chain[idx], Y_chain[idx], adj)

{'average': 0.2709534644385706,
 'direct_effect': 0.24466417596799514,
 'spillover_effect': 0.09915752331446545,
 'psi_1_gamma': 0.3438216992824606,
 'psi_0_gamma': 0.09915752331446545,
 'psi_zero': 0.0}